<a href="https://colab.research.google.com/github/JINU6497/JINUrepo/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC_%EB%B0%8F_Pre_Trained_Embedding_Vector%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_Vocabulary_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

TEXT=data.Field(batch_first=True,
                fix_length=500,
                tokenize=str.split,
                pad_first=True,
                pad_token='[PAD]',
                unk_token='[UNK]'
                )
# fix_label: Sentence의 길이를 미리 제한하는 옵션
# tokenize: Tokenize를 설정하는 옵션. 기본값은 띄어쓰기 기반의 파이썬의 string.split 함수
# pad_first: fix_length 대비 짧은 문장의 경우에는 padding 해줘야 하는데, padding을 앞에서 해 줄 것인지에 대한 옵션
# pad_token: 설정한 padding에 대한 특수 Token 설정
# unk_token: Token dictionary에 없는 token이 나왔을 경우 해당 token을 표현하는 특수 token

LABEL=data.LabelField(dtype=torch.float)
# dtype: 가져올 데이터에 대한 Type 설정 옵션

train_data, test_data = datasets.IMDB.splits(text_field=TEXT, label_field=LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:00<00:00, 89.0MB/s]


In [6]:
# Data length
print(f'Train Data Length: {len(train_data.examples)}')
print(f'Test Data Length: {len(test_data.examples)}')

# Data field
print(train_data.fields)

# Data Sample
print('---- Data Sample ----')
print('Input: ')
print(''.join(vars(train_data.examples[1])['text']),'\\n')
print('Label: ')
print(vars(train_data.examples[1])['label'])

Train Data Length: 25000
Test Data Length: 25000
{'text': <torchtext.legacy.data.field.Field object at 0x7fa09c728050>, 'label': <torchtext.legacy.data.field.LabelField object at 0x7fa0964297d0>}
---- Data Sample ----
Input: 
AgreatBritishIndymovie!Fantasticchemistrybetweenthe3maincharactersmakeforsomehilariousdrug-fuelledsetpiecesthatCheechandChongwouldbeproudof.GreattoseePhilDanielsbackonthebigscreen(evenifhehasswappedsidessinceQuadrophenia!)andGaryStretchissurprisinglygoodandatreatfortheladies!Lovedthefinalfightscenewithit'snodtoZuluandnowIknowwhathappenedtoArthurBrownafterhesethimselfonfireonTopofthePops!...he'snotacting....hereallyisabona-fideBritishhippie!!!Youdon'thavetobeabikertoenjoythisandit'sstraightintomyFridaynightpost-pubrepeatviewingcollection.<br/><br/>Givethisfilmagoandyouwon'tbedisappointed. \n
Label: 
pos


data.examples를 이용하여 데이터의 개수를 확인할 수 있고, vars()함수를 이용하여 데이터의 값을 직접 확인. 이때 Text data는 Tokenize를 하기 전에 Data를 살펴보고 cleansing작업을 해야 한다. 이는 다음과 같이 진행한다. (이는 Field의 preprocessing옵션을 이용하여도 미리 작업 가능)

In [18]:
import re

def PreProcessingText(input_sentence):
  input_sentence = input_sentence.lower()  # 소문자화
  input_sentence = re.sub('<{^>]*>', repl='', string=input_sentence)  # "<br />" 처리
  input_sentence = re.sub('[!"#$%&\\()*+,-./:;<=>?@[\\\\]^_{|}~]',repl='',string=input_sentence)  # 특수 문자 처리("'" 제외)
  input_sentence = re.sub('\\s+',repl='',string=input_sentence)  # 연속된 띄어쓰기 처리

  if input_sentence:
    return input_sentence

for example in train_data.examples:
  vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split( )

for example in test_data.examples:
  vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split( )


In [23]:
# 주어진 Data를 이용하여 Token Vocabulary를 만드는 과정

# pre-trained
TEXT.build_vocab(train_data, min_freq =2,  # min_freq: Vocab에 해당하는 Token에 최소한으로 등장하는 횟수에 제한을 두는 것
                 max_size = None,  # max_size: Token의 최소 등장 횟수로 Vocab의 Size조절을 하는 것 외에 전체 Vocab Size 자체에도 제한을 두는 것
                 vectors = "glove.6B.300d"  # vectors: pre-trained vector를 가져와 Vocab에 세팅. 원하는 Embedding을 정해 string 형태로 설정
                 )

LABEL.build_vocab(train_data)

100%|█████████▉| 399999/400000 [00:47<00:00, 8400.98it/s]


In [25]:
# Vocab에 대한 정보 확인하기

print(f'Vocab Size:{len(TEXT.vocab)}')

print('Vocab examples: ')

for idx,(k,v) in enumerate(TEXT.vocab.stoi.items()):
  if idx >=10:
    break
  print('\\t',k,v)

print('--------------------------------------------')

# Label에 대한 정보 확인

print(f'Label Size:{len(LABEL.vocab)}')

print('Label Examples: ')
for idx,(k,v) in enumerate(LABEL.vocab.stoi.items()):
  print('\\t',k,v)

# Check embedding vector

print(TEXT.vocab.vectors.shape)

Vocab Size:96
Vocab examples: 
\t [UNK] 0
\t [PAD] 1
\t howhasthispieceofcrapstayedontvthislong?it'sterrible.itmakesmewanttoshootsomeone.it'ssofakethatitisactuallyworsethana1940ssci-fimovie.i'dratherhaveastrokethanwatchthisnonsense.irememberwatchingitwhenitfirstcameout.ithought,heythiscouldbeinteresting,thenifoundouthowabsolutely,insanely,ridiculouslystupiditreallywas.itwassobadthatiactuallytookoutmypocketknifeandstuckmyhandtothetable.<br/><br/>pleasepeople,stopwatchingthisandallotherrealityshows,they'rethetrashthatisjammingthenetworksandcancelingqualityprogrammingthatrequiressomethoughttocreate. 2
\t thisshowcomesupwithinterestinglocationsasfastasthetravelchannel.itisbilledasrealitybutinactualityitispureprimetimesoapopera.it'striestouseexoticlocalesasafacadetobringpeopleintoaphonycontest&thenproceedstohookviewersonthecontestantssoapoperastyle.<br/><br/>italsoborrowsfromanearlycbsgameshowpioneer-beattheclock-byinventingsituationsforitscontestantstotry&overcome.thenitrewardsthewinnermon

In [26]:
# Validation Set 구분과 Iterator를 이용하여 Batch Data를 만들면 모델 학습을 위한 데이터 설정은 끝

import random
train_data, valid_data = train_data.split(random_state=random.seed(0),split_ratio=0.8) # random.seed(): 무작위가 포함된 공정의 완전한 재현을 가능하게끔 만들어주는 것

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(datasets=(train_data, valid_data, test_data),
                                                                           batch_size=30, device=device)